In [6]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)

filter_label = catalog.load('params:openaire_fetch_options.filter_label')
df = catalog.load('raw/openaire/researchproduct#parquet')

[02/28/25 10:36:06] INFO     Loading data from params:openaire_fetch_options.filter_label       ]8;id=87695;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=218269;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from raw/openaire/researchproduct#parquet             ]8;id=272508;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=991115;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185373 entries, 0 to 185372
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   author              185305 non-null  object
 1   openAccessColor     53613 non-null   object
 2   publiclyFunded      121560 non-null  object
 3   type                185373 non-null  object
 4   language            185373 non-null  object
 5   subjects            183336 non-null  object
 6   mainTitle           185373 non-null  object
 7   description         183327 non-null  object
 8   publicationDate     185361 non-null  object
 9   publisher           76578 non-null   object
 10  source              68904 non-null   object
 11  format              170917 non-null  object
 12  bestAccessRight     99741 non-null   object
 13  container           48356 non-null   object
 14  id                  185373 non-null  object
 15  originalId          185373 non-null  object
 16  in

In [8]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'source', 'format', 'bestAccessRight', 'container', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal',
       'contributor', 'pid', 'country', 'contactPerson', 'embargoEndDate',
       'coverage', 'version', 'size', 'filter_applied'],
      dtype='object')

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [9]:
def land_openaire_map_researchproduct_subject(df: pd.DataFrame)-> pd.DataFrame:

    expected_columns = [
        'id',
        'subjects',
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[expected_columns].copy()
    df = df.convert_dtypes()
    df.reset_index(drop=True, inplace=True)

    ## subjects
    df_researchproduct2subject = df.explode('subjects').reset_index(drop=True)
    df_researchproduct2subject = df_researchproduct2subject[['id','subjects']]
    df_subjects = pd.json_normalize(df_researchproduct2subject['subjects']).reset_index(drop=True)
    df_researchproduct2subject = pd.concat([df_researchproduct2subject.drop(columns=['subjects']), df_subjects], axis=1)

    df_researchproduct2subject['load_datetime'] = date.today()

    return df_researchproduct2subject


In [10]:
df_researchproduct2subject = land_openaire_map_researchproduct_subject(df)

In [11]:
df_researchproduct2subject

,id,provenance,subject.scheme,subject.value,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,ganancia diaria de peso,2025-02-28
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,cabritos,2025-02-28
2,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,kids,2025-02-28
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,goats,2025-02-28
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,Saanen,2025-02-28
...,...,...,...,...,...
1441917,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,NaN,NaN,NaN,2025-02-28
1441918,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,NaN,keyword,Coreia do Sul. Papel regional e internacional....,2025-02-28
1441919,portalunbbra::fe22079ac12e4b48e955751af4ce027b,NaN,NaN,NaN,2025-02-28
1441920,revistasumft::106b3b4f6115356de22e7aaf07960b34,NaN,NaN,NaN,2025-02-28
